### **Word2Vec Encoding Method**

In [12]:
import pandas as pd

# Load the dataset
# Replace with the actual path to your CSV file
df = pd.read_csv('/content/cleaned_dataset.csv')

# Display the first few rows to verify
print("Original DataFrame:")
print(df.head())




Original DataFrame:
   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  \
0  retweet as a woman you should not complain abo...   
1  retweet boy dats coldtyga down bad for cuffin ...   
2  retweet dawg retweet you ever fuck a bitch and...   
3                     retweet she look like a tranny   
4  retweet the shit you hear about me might be tr...   

                                        tweet_tokens  
0  ['retweet', 'woman', 'complain', 'cleaning', '...  
1  ['retweet', 'boy', 'dat', 'coldtyga', 'bad', '...  
2  ['retweet', 'dawg', 'retweet', 'ever

In [13]:
import ast

# Convert string representation of list to actual list
df['tweet_tokens'] = df['tweet_tokens'].apply(ast.literal_eval)

# Display the first few rows to verify
print("\nDataFrame after converting tweet_tokens to lists:")
print(df.head())



DataFrame after converting tweet_tokens to lists:
   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  \
0  retweet as a woman you should not complain abo...   
1  retweet boy dats coldtyga down bad for cuffin ...   
2  retweet dawg retweet you ever fuck a bitch and...   
3                     retweet she look like a tranny   
4  retweet the shit you hear about me might be tr...   

                                        tweet_tokens  
0  [retweet, woman, complain, cleaning, house, an...  
1  [retweet, boy, dat, coldtyga, bad, cuffin, dat...  
2  [retw

In [14]:
from gensim.models import Word2Vec

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=df['tweet_tokens'], vector_size=100, window=5, min_count=1, workers=4)

# Get the vector size
vector_size = word2vec_model.wv.vector_size



In [15]:
import numpy as np

def document_vector(doc):
    # Remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    # If the document is empty, return a zero vector
    if not doc:
        return np.zeros(vector_size)
    # Calculate the mean vector
    return np.mean(word2vec_model.wv[doc], axis=0)

# Apply the function to transform tweet_tokens
X_vectors = df['tweet_tokens'].apply(document_vector).tolist()
X_vectors = np.array(X_vectors)

# Display the first few rows to verify
print("\nVectorized DataFrame:")
print(X_vectors)
print(X_vectors.shape)




Vectorized DataFrame:
[[-0.12281501  0.55791259  0.03236658 ... -0.61398578  0.25563625
  -0.69630682]
 [-0.06064844  0.46781531 -0.01103297 ... -0.48454204  0.24361667
  -0.60603142]
 [-0.08893751  0.55862242 -0.02659687 ... -0.55425692  0.28885379
  -0.69526631]
 ...
 [ 0.01220226  0.4365465  -0.03993332 ... -0.50972438  0.41628036
  -0.94992763]
 [-0.01496591  0.36141133 -0.01601156 ... -0.36901939  0.18593688
  -0.46845421]
 [-0.06536717  0.23087177  0.03120324 ... -0.26092887  0.09738781
  -0.27045333]]
(24783, 100)


In [16]:
# Combine Word2Vec features with other features
other_features = df[['hate_speech', 'offensive_language', 'neither']]
X_combined = np.hstack((other_features, X_vectors))

# Define the target variable
y = df['class']

# Display the first few rows to verify
print("\nCombined DataFrame:")
print(X_combined)
print(X_combined.shape)


Combined DataFrame:
[[ 0.          0.          3.         ... -0.61398578  0.25563625
  -0.69630682]
 [ 0.          3.          0.         ... -0.48454204  0.24361667
  -0.60603142]
 [ 0.          3.          0.         ... -0.55425692  0.28885379
  -0.69526631]
 ...
 [ 0.          3.          0.         ... -0.50972438  0.41628036
  -0.94992763]
 [ 0.          6.          0.         ... -0.36901939  0.18593688
  -0.46845421]
 [ 0.          0.          3.         ... -0.26092887  0.09738781
  -0.27045333]]
(24783, 103)
